In [1]:
import tensorflow as tf
import keras
import numpy as np
from pathlib import Path
import sys

project_root = Path("../../").resolve()
sys.path.append(str(project_root))
from scripts.yolo_finetuning.yolo_inference import YOLOInference


device = "cuda" if tf.config.list_physical_devices("GPU") else "cpu"

# Set memory growth for GPU
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

print(f"GPUs available: {len(tf.config.list_physical_devices('GPU'))}")

# Initialize YOLO preprocessor as specified in MobileNet_training.md
yolo_model_path = "../../notebooks/yolo/best_so_far.pt"
if Path(yolo_model_path).exists():
    try:
        yolo_inference = YOLOInference(yolo_model_path, conf_threshold=0.25, iou_threshold=0.45)
        print(f"✓ YOLO model loaded: {yolo_model_path}")
        use_yolo_preprocessing = True
    except Exception as e:
        print(f"⚠ YOLO loading failed: {e}")
        use_yolo_preprocessing = False
else:
    print(f"⚠ YOLO model not found at: {yolo_model_path}")
    use_yolo_preprocessing = False

2025-06-28 13:56:08.971519: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-28 13:56:08.979508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-28 13:56:08.988755: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-28 13:56:08.991593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-28 13:56:08.998778: I tensorflow/core/platform/cpu_feature_guar

✓ YOLO inference imported successfully
GPUs available: 0
✓ YOLO model loaded: ../../notebooks/yolo/best_so_far.pt


2025-06-28 13:56:10.793500: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-06-28 13:56:10.793529: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:135] retrieving CUDA diagnostic information for host: 2a01cb0001104f00845b1dbbc3600f10.ipv6.abo.wanadoo.fr
2025-06-28 13:56:10.793543: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:142] hostname: 2a01cb0001104f00845b1dbbc3600f10.ipv6.abo.wanadoo.fr
2025-06-28 13:56:10.793726: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:166] libcuda reported version is: 570.153.2
2025-06-28 13:56:10.793745: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:170] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  570.153.02  Release Build  (dvs-builder@U22-A23-20-3)  Tue May 13 16:34:58 UTC

In [2]:
# YOLO Preprocessing Functions (as specified in MobileNet_training.md)

def preprocess_image_with_yolo(image_path, target_size=(224, 224)):
    """
    Apply YOLO preprocessing to get footprint crops, with center crop fallback.
    This implements the requirement from MobileNet_training.md.
    """
    if not use_yolo_preprocessing:
        # Fallback: center crop
        from PIL import Image
        img = Image.open(image_path)
        width, height = img.size
        size = min(width, height)
        left, top = (width - size) // 2, (height - size) // 2
        cropped = img.crop((left, top, left + size, top + size))
        resized = cropped.resize(target_size)
        return np.array(resized)
    
    try:
        # Use YOLO detection as specified in requirements
        result = yolo_inference.infer_and_get_best_crop(str(image_path))
        
        if result is not None:
            bbox, cropped_image = result
            # Resize to target size for MobileNet
            from PIL import Image
            cropped_pil = Image.fromarray(cropped_image)
            resized_image = cropped_pil.resize(target_size)
            return np.array(resized_image)
        else:
            # Fallback to center crop if YOLO fails
            from PIL import Image
            img = Image.open(image_path)
            width, height = img.size
            size = min(width, height)
            left, top = (width - size) // 2, (height - size) // 2
            cropped = img.crop((left, top, left + size, top + size))
            resized = cropped.resize(target_size)
            return np.array(resized)
            
    except Exception as e:
        print(f"YOLO processing failed for {image_path}: {e}")
        # Fallback to center crop
        from PIL import Image
        img = Image.open(image_path)
        width, height = img.size
        size = min(width, height)
        left, top = (width - size) // 2, (height - size) // 2
        cropped = img.crop((left, top, left + size, top + size))
        resized = cropped.resize(target_size)
        return np.array(resized)

def create_yolo_preprocessed_dataset(data_dir, batch_size=16, validation_split=None):
    """
    Create dataset with YOLO preprocessing as specified in MobileNet_training.md.
    """
    from pathlib import Path
    
    data_path = Path(data_dir)
    image_paths = []
    labels = []
    class_names = []
    
    # Collect images and labels
    for class_dir in sorted(data_path.iterdir()):
        if class_dir.is_dir():
            class_names.append(class_dir.name)
            class_idx = len(class_names) - 1
            
            for img_path in class_dir.glob("*.jpg"):
                image_paths.append(str(img_path))
                labels.append(class_idx)
    
    print(f"Found {len(image_paths)} images across {len(class_names)} classes")
    print(f"Classes: {class_names}")
    
    def preprocess_function(image_path, label):
        """Apply YOLO preprocessing."""
        processed_image = preprocess_image_with_yolo(image_path.numpy().decode('utf-8'))
        return processed_image.astype(np.float32), label
    
    # Create TensorFlow dataset
    path_ds = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    
    # Apply YOLO preprocessing
    dataset = path_ds.map(
        lambda path, label: tf.py_function(
            preprocess_function, 
            [path, label], 
            [tf.float32, tf.int32]
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    
    # Set shapes and normalize
    dataset = dataset.map(lambda img, label: (tf.reshape(img, [224, 224, 3]), label))
    dataset = dataset.map(lambda img, label: (img / 255.0, label))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    
    return dataset, class_names

print("✓ YOLO preprocessing functions ready")

✓ YOLO preprocessing functions ready


In [3]:
# Load OAT datasets with YOLO preprocessing (as specified in MobileNet_training.md)
print("Loading OAT datasets with YOLO preprocessing...")

if use_yolo_preprocessing:
    print("Using YOLO preprocessing for all datasets")
    
    # Use YOLO preprocessing
    train_ds, oat_class_names = create_yolo_preprocessed_dataset(
        "../../data/OpenAnimalTracks_spokay/cropped_imgs/train",
        batch_size=16
    )
    
    val_ds, _ = create_yolo_preprocessed_dataset(
        "../../data/OpenAnimalTracks_spokay/cropped_imgs/val",
        batch_size=16
    )
    
    test_ds, _ = create_yolo_preprocessed_dataset(
        "../../data/OpenAnimalTracks_spokay/cropped_imgs/test",
        batch_size=16
    )
    
else:
    print("Fallback: Using standard image loading")
    
    # Fallback to standard loading
    train_ds = keras.preprocessing.image_dataset_from_directory(
        "../../data/OpenAnimalTracks_spokay/cropped_imgs/train",
        image_size=(224, 224),
        batch_size=16,
    )
    test_ds = keras.preprocessing.image_dataset_from_directory(
        "../../data/OpenAnimalTracks_spokay/cropped_imgs/test",
        image_size=(224, 224),
        batch_size=16,
    )
    val_ds = keras.preprocessing.image_dataset_from_directory(
        "../../data/OpenAnimalTracks_spokay/cropped_imgs/val",
        image_size=(224, 224),
        batch_size=16,
    )
    oat_class_names = train_ds.class_names

print(f"OAT dataset loaded: {len(oat_class_names)} classes")
print(f"Classes: {oat_class_names}")

Loading OAT datasets with YOLO preprocessing...
Using YOLO preprocessing for all datasets
Found 2514 images across 18 classes
Classes: ['beaver', 'black_bear', 'bob_cat', 'coyote', 'elephant', 'goose', 'gray_fox', 'horse', 'lion', 'mink', 'mouse', 'muledeer', 'otter', 'raccoon', 'rat', 'skunk', 'turkey', 'western_grey_squirrel']
Found 346 images across 18 classes
Classes: ['beaver', 'black_bear', 'bob_cat', 'coyote', 'elephant', 'goose', 'gray_fox', 'horse', 'lion', 'mink', 'mouse', 'muledeer', 'otter', 'raccoon', 'rat', 'skunk', 'turkey', 'western_grey_squirrel']
Found 719 images across 18 classes
Classes: ['beaver', 'black_bear', 'bob_cat', 'coyote', 'elephant', 'goose', 'gray_fox', 'horse', 'lion', 'mink', 'mouse', 'muledeer', 'otter', 'raccoon', 'rat', 'skunk', 'turkey', 'western_grey_squirrel']
OAT dataset loaded: 18 classes
Classes: ['beaver', 'black_bear', 'bob_cat', 'coyote', 'elephant', 'goose', 'gray_fox', 'horse', 'lion', 'mink', 'mouse', 'muledeer', 'otter', 'raccoon', 'rat

In [4]:
mobilenet = keras.applications.MobileNetV3Small(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet",
)

# Initially freeze the base model
mobilenet.trainable = False

# Data augmentation
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1),
    keras.layers.RandomContrast(0.1),
])

# Build the complete model
model = keras.Sequential([
    data_augmentation,
    mobilenet,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(256, activation="relu", kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(18, activation="softmax"),
])

In [5]:

model.compile(optimizer="adamax", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)])

# Save the model
model.save("mobilenet.keras")

Epoch 1/50


ValueError: Cannot take the length of shape with unknown rank.

In [ ]:
mobilenet_loaded = keras.models.load_model("mobilenet.keras")
mobilenet.trainable = True

fine_tune_at = len(mobilenet.layers) - 20

for layer in mobilenet.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5), 
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [40]:
model.fit(train_ds, epochs=10, validation_data=val_ds, callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)])

Epoch 1/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.4119 - loss: 2.9213 - val_accuracy: 0.4306 - val_loss: 2.8900
Epoch 2/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4416 - loss: 2.7654 - val_accuracy: 0.4306 - val_loss: 2.9450
Epoch 3/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4766 - loss: 2.6233 - val_accuracy: 0.4335 - val_loss: 2.9369
Epoch 4/10
158/158 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4886 - loss: 2.5909 - val_accuracy: 0.4393 - val_loss: 2.9169


In [ ]:
# Load Real dataset with YOLO preprocessing (as specified in MobileNet_training.md)
print("Loading Real dataset with YOLO preprocessing...")

if use_yolo_preprocessing:
    print("Using YOLO preprocessing for Real dataset")
    
    # Load full dataset first to get all images
    real_ds_full, real_class_names = create_yolo_preprocessed_dataset(
        "../../data/dataset_no_oat_downsample_spokay",
        batch_size=32
    )
    
    # Split into train and validation
    train_size = int(0.8 * len(list(real_ds_full)))
    val_size = len(list(real_ds_full)) - train_size
    
    real_train_ds = real_ds_full.take(train_size)
    real_val_ds = real_ds_full.skip(train_size)
    
else:
    print("Fallback: Using standard image loading for Real dataset")
    
    real_ds = keras.preprocessing.image_dataset_from_directory(
        "../../data/dataset_no_oat_downsample_spokay",
        image_size=(224, 224),
        batch_size=32,
    )
    
    real_class_names = real_ds.class_names
    
    # Split into train and validation
    train_size = int(0.8 * len(real_ds))
    val_size = len(real_ds) - train_size
    
    real_train_ds = real_ds.take(train_size)
    real_val_ds = real_ds.skip(train_size)

print(f"Real dataset loaded: {len(real_class_names)} classes")
print(f"Classes: {real_class_names}")
print(f"Dataset split - Training batches: {train_size}, Validation batches: {val_size}")

In [44]:

train_size = int(0.8 * len(real_ds))
val_size = len(real_ds) - train_size

real_train_ds = real_ds.take(train_size)
real_val_ds = real_ds.skip(train_size)

print(f"Dataset d'entraînement: {train_size} batches")
print(f"Dataset de validation: {val_size} batches")


Dataset d'entraînement: 28 batches
Dataset de validation: 7 batches


In [ ]:
print("Début du fine-tuning...")

history = model.fit(
    real_train_ds,
    epochs=60,
    validation_data=real_val_ds,
    callbacks=[
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-7)
    ]
)

model.save("mobilenet_finetuned.keras")
print("Modèle fine-tuné sauvegardé sous 'mobilenet_finetuned.keras'")

In [ ]:
# Évaluation du modèle fine-tuné
import matplotlib.pyplot as plt

# Afficher les courbes d'apprentissage
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Perte du modèle')
plt.xlabel('Époque')
plt.ylabel('Perte')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Précision du modèle')
plt.xlabel('Époque')
plt.ylabel('Précision')
plt.legend()

plt.tight_layout()
plt.show()

# Évaluer sur l'ensemble de validation
val_loss, val_accuracy = model.evaluate(real_val_ds)
print(f"Précision finale sur les données de validation: {val_accuracy:.4f}")
print(f"Perte finale sur les données de validation: {val_loss:.4f}")